<h1 style="text-align: center;">Data Visualization Project</h1>
<h2 style="text-align: center;">Fashion-MNIST Dataset</h2>

# Preface
* Fashion-MNIST is a dataset containing 70,000 samples, 60,000 for the training set and 10,000 for the test set.
* Each sample is a 28x28 (784 pixels) grayscale image of a certain fashion item.
* The data contains a column with 10 labels, making this a **multiclass classification** problem.
In other words, this is a **supervised learning** task.
* The model will be trained using all available data and run without learning anymore, also known as **offline/batch learning**.
* ***Main objective:*** Find the best algorithm and model parameters that classify the unseen images correctly.

In [ ]:
# common imports
import numpy as np
import pandas as pd
import seaborn as sns
import joblib
import matplotlib.pyplot as plt
%matplotlib inline

# machine learning imports
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, VotingClassifier
from xgboost import XGBClassifier, cv, DMatrix
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict, train_test_split
from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn import metrics
from sklearn import clone  # 'clone' constructs a new unfitted estimator with the same parameters
from sklearn.dummy import DummyClassifier
from scipy.ndimage.interpolation import shift

# global variables
label_dict = {0: "0 = T-shirt/top", 1: "1 = Trouser", 2: "2 = Pullover", 3: "3 = Dress", 4: "4 = Coat",
              5: "5 = Sandal", 6: "6 = Shirt", 7: "7 = Sneaker", 8: "8 = Bag", 9: "9 = Ankle boot"}

# display setup
plt.style.use('seaborn')  # for plots

# 1. Getting the Data

In [ ]:
# read the csv file
train_set = pd.read_csv(r"FMNIST/fashion-mnist_train.csv")
test_set = pd.read_csv(r"FMNIST/fashion-mnist_test.csv")

In [ ]:
# display the first 5 rows for a quick look
train_set.head()

In [ ]:
# display the last 5 rows for a quick look
train_set.tail()

In [ ]:
# DataFrame shape (rows, columns)
print("Training Set:", train_set.shape)
print("Test Set:", test_set.shape)

In [ ]:
# description of data
train_set.info()

In [ ]:
# summary of the numerical attributes
train_set.describe()

In [ ]:
# maximum pixel value
train_set.describe().loc['max'].max()

In [ ]:
# minimum pixel value
train_set.describe().loc['min'].max()

> ### Features in the DataFrame:
> There are 785 columns, one for the labels and 784 for the pixels (one for each pixel).
>> Labels:
> - 0 = T-shirt/top
> - 1 = Trouser
> - 2 = Pullover
> - 3 = Dress
> - 4 = Coat
> - 5 = Sandal
> - 6 = Shirt
> - 7 = Sneaker
> - 8 = Bag
> - 9 = Ankle boot

In [ ]:
# number of instances for each category
train_set["label"].value_counts().sort_index()

> #### Initial observations:
* Each category has an equal amount of samples in the training set, making this a **balanced classification** task.
* Classes and pixel values are integers.
* The pixel range is [0, 255]. Some feature columns have a smaller maximum value or a
greater minimum value. This means that the values range is smaller for some pixels.

# 2. Understanding and Visualizing the Data
> ##### *The motivation for this section is to gain more insights.*

> The data was split in advance and the images are already the same aspect ratio.
Let's create a copy of the data to prevent accidentally harming the training set.

In [ ]:
# deep copy of the training set
fmnist = train_set.copy()

In [ ]:
fmnist.head(2)

In [ ]:
# check for missing values
# np.isnan checks if the element is is not a number
# df.values returns a numpy array containing the data without index or column names
# sum() returns the absolute amount missing
np.isnan(fmnist.values).sum()

> #### Observations:
> * There are no missing values in the training set.

> ### Exploring Color Transformations on a Sample Image

In [ ]:
# sample image
some_sample = fmnist.drop('label', axis=1).iloc[0]  # get sample
some_sample = np.array(some_sample)  # convert to array
some_sample_img = some_sample.reshape(28, 28)  # reshape array

# convert sample image from grayscale to black and white
# in the fashion mnist dataset- white pixels are 0 and black pixels are 255
# the images are grayscale, so values (0,255) are different intensities of gray
some_sample_bin = some_sample.copy()  # deep copy of the sample image
some_sample_bin[some_sample_bin > 0] = 1  # convert gray intensities to black
some_sample_bin_img = some_sample_bin.reshape(28, 28)  # reshape array

fig, dx = plt.subplots(2, 2, figsize=(12, 7))

# plot grayscale sample image and pixel value occurrences
dx[0, 0].imshow(some_sample_img)
dx[0, 0].axis('off')
dx[0, 0].set_title("Original Grayscale Sample", size=15)
dx[0, 1].hist(some_sample, bins=50)
dx[0, 1].set_title("Grayscale Sample Pixel Value Occurrences", size=15)
dx[0, 1].set_xlabel("Pixel Value")
dx[0, 1].set_ylabel("Count")

# plot black and white sample image and pixel value occurrences
dx[1, 0].imshow(some_sample_bin_img)
dx[1, 0].axis('off')
dx[1, 0].set_title("Black and White Sample", size=15)
dx[1, 1].hist(some_sample_bin, bins=50)
dx[1, 1].set_title("Black and White Sample Pixel Value Occurrences", size=15)
dx[1, 1].set_xlabel("Pixel Value")
dx[1, 1].set_ylabel("Count")

plt.tight_layout()
plt.show()

> #### Observations:
* The most common pixel value in the original grayscale image is white (0) and in the binary
image is black (1 = 255 in grayscale).
* Aside from a few pixels that are detached from the clothing, the black and white pullover resembles
the original grayscale shape.
* Training models on both options and comparing the results can help determine if transforming the
images to black and white is preferable.

> ### Class Comparison

In [ ]:
# plot image for each category
fig = plt.figure(figsize=(10, 7))
rows = 2
columns = 5
# use groupby to locate an instance for each label
label_groups = fmnist.groupby('label')
# add image in each iteration
for i in range(rows * columns):
    curr = label_groups.get_group(i)[:1]  # get group
    curr_img = curr.drop('label', axis=1).to_numpy().reshape(28, 28)  # convert to reshaped array
    fig.add_subplot(rows, columns, i + 1)
    plt.imshow(curr_img)
    plt.axis('off')  # remove grid
    plt.title(label_dict[i])  # use dictionary to add subplot title

fig.suptitle("Fashion-MNIST Samples", size=25)
plt.tight_layout()
plt.show()

In [ ]:
# plot pixel value occurrences for each class

fig, dx = plt.subplots(2, 5, figsize=(12, 7), sharey='all')
i = 0  # current group label
mean_values = []
plt.setp(dx, xticks=np.arange(0, 256, step=85))  # set x axis values

for row in range(2):
    for col in range(5):
        pixels = np.array(label_groups.get_group(i).drop(['label'], axis=1))  # get group and convert to array
        mean_values.append(pixels.mean())  # calculate mean pixel value and add to list (for next plot)
        dx[row, col].hist(pixels.reshape(-1))  # add histogram in each iteration, -1 reshapes to length of array
        dx[row, col].set_title(label_dict[i], size=15)  # use dictionary to add subplot title
        i = i + 1  # next group

fig.suptitle("Pixel Occurrences per Class", size=25)
plt.tight_layout()
plt.show()

In [ ]:
# plot mean values calculated in previous cell
plt.figure(figsize=(12, 5))
sns.barplot(x=np.arange(10), y=mean_values)  # x axis for classes, y axis for mean values
plt.xticks(np.arange(10), labels=label_dict.values())  # use dictionary to set x axis values
plt.xlabel("Class", size=15)
plt.ylabel("Mean", size=15)
plt.title("Pixel Occurrences per Class Mean", size=25)
plt.tight_layout()
plt.show()

> #### Observations:
* All classes have a majority of white pixel values (0), with the rest scattered in the remaining range.
* Aside from the white pixel value, quite a few of the histograms displaying pixel occurrences look like
they have a somewhat normal distribution. They are tail-heavy, extending farther to the right.
* Any shoe type (classes 5, 7, 9) have the most white pixels, with sandal (class 5) containing the most.
This is emphasized most in the pixel occurrences mean plot.
* Coats (class 4) and pullovers (class 2) have the highest pixel mean. When looking at the sample
images it is noticeable that they fill most of the diagram.
* The t-shirt/top (class 0) and shirt (class 6) have extremely similar pixel occurrences.
Had the class labels been removed, the graphs would be nearly indistinguishable. We can assume that
the models will make mistakes when predicting instances from these two classes.

> ### Analyze and Compare Sample Images of T-shirt/top and Shirt Classes

In [ ]:
# t-shirt/top sample image (class 0)
top = label_groups.get_group(0).drop('label', axis=1).iloc[0]  # get t-shirt/top sample
top = np.array(top)  # convert to array
top_img = top.reshape(28, 28)  # reshape array

# convert t-shirt/top sample image from grayscale to black and white
top_bin = top.copy()  # deep copy of the t-shirt/top sample image
top_bin[top_bin > 0] = 1  # convert gray intensities to black
top_bin_img = top_bin.reshape(28, 28)  # reshape array

# shirt sample image (class 6)
shirt = label_groups.get_group(6).drop('label', axis=1).iloc[0]  # get t-shirt/top sample
shirt = np.array(shirt)  # convert to array
shirt_img = shirt.reshape(28, 28)  # reshape array

# convert shirt sample image from grayscale to black and white
shirt_bin = shirt.copy()  # deep copy of the shirt sample image
shirt_bin[shirt_bin > 0] = 1  # convert gray intensities to black
shirt_bin_img = shirt_bin.reshape(28, 28)  # reshape array

In [ ]:
fig, dx = plt.subplots(4, 2, figsize=(10, 10))

# plot grayscale t-shirt/top sample image and pixel value occurrences
dx[0, 0].imshow(top_img)
dx[0, 0].axis('off')
dx[0, 0].set_title("Grayscale T-shirt/top", size=15)
dx[0, 1].hist(top)
dx[0, 1].set_title("Grayscale T-shirt/top Pixel Value Occurrences", size=15)
dx[0, 1].set_xlabel("Pixel Value")
dx[0, 1].set_ylabel("Count")
# plot black and white t-shirt/top sample image and pixel value occurrences
dx[1, 0].imshow(top_bin_img)
dx[1, 0].axis('off')
dx[1, 0].set_title("Black and White T-shirt/top", size=15)
dx[1, 1].hist(top_bin)
dx[1, 1].set_title("Black and White T-shirt/top Pixel Value Occurrences", size=15)
dx[1, 1].set_xticks([0, 1])
dx[1, 1].set_xlabel("Pixel Value")
dx[1, 1].set_ylabel("Count")

# plot grayscale shirt sample image and pixel value occurrences
dx[2, 0].imshow(shirt_img)
dx[2, 0].axis('off')
dx[2, 0].set_title("Grayscale Shirt", size=15)
dx[2, 1].hist(shirt)
dx[2, 1].set_title("Grayscale Shirt Pixel Value Occurrences", size=15)
dx[2, 1].set_xlabel("Pixel Value")
dx[2, 1].set_ylabel("Count")
# plot black and white shirt sample image and pixel value occurrences
dx[3, 0].imshow(shirt_bin_img)
dx[3, 0].axis('off')
dx[3, 0].set_title("Black and White Shirt", size=15)
dx[3, 1].hist(shirt_bin)
dx[3, 1].set_title("Black and White Shirt Pixel Value Occurrences", size=15)
dx[3, 1].set_xticks([0, 1])
dx[3, 1].set_xlabel("Pixel Value")
dx[3, 1].set_ylabel("Count")

plt.tight_layout()
plt.show()

> #### Observations:
* The main difference between the classes are the pixels representing the sleeves.
The t-shirt/top has short sleeves and the shirt has long sleeves.
* Converting the images to binary shows where the mix-up between the classes might be.
The t-shirt has gray pixels that aren't seen in the grayscale image, but stand out in the
black and white image. The unseen gray pixels appear where the long sleeves would be-
had this been a shirt.
* The pixel value occurrences are similar when comparing the black and white images and are
spread out differently in the grayscale images.

# 3. Data Cleaning

In [ ]:
# clean copy of the training set
df = train_set.copy()

In [ ]:
# separate features from target values

# drop- creates a copy without changing the training set
X_train = df.drop('label', axis=1)

# create a deep copy of the target values
y_train = df['label'].copy()

> ### Custom Transformer:
> In section 2, I evaluated samples in grayscale and in black and white.
> I assumed that converting images to black and white is a worthwhile transformation to look into
> and could improve the ML algorithms.
>
> The following custom transformer automates this process on the entire dataset:
>
>> The ColorConverter transformer contains the hyperparameter "is_binary". It is
> set by default to 'False', meaning the samples remain in the original grayscale format.
> However, the hyperparameter can be set to 'True' which will convert the images to black
> and white.
>>
>> Adding this transformer will allow to easily switch between the two options during
> model training, and determine which one is better.

In [ ]:
# custom transformer for converting images to black and white

# BaseEstimator for enabling hyperparameters
# TransformerMixin adds fit_transform method
class ColorConverter(BaseEstimator, TransformerMixin):

    def __init__(self, is_binary=False):
        self.is_binary = is_binary

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if self.is_binary:
            X_binary = X.copy() # deep copy to avoid harming the original data
            X_binary[X_binary > 0] = 1 # convert grayscale intensities to black
            return X_binary
        else:
            return X

> ### Feature Scaling
>
> Although the pixel values are in a known range [0,255], scaling the data can make a crucial difference
(especially if the learning algorithm relies on calculating distances).
>
>> Why is this important?
* Models can't differentiate feature importance the same way humans can.
A training algorithm may assume that a feature containing large numbers is more important than features
with smaller numbers- which might not be the case.
* Some algorithms converge much faster when features are scaled (i.e. Gradient Descent).
* There are ML algorithms that make assumptions on the data (i.e. PCA assumes the data is centered around
the origin).


> Chosen feature scale:
>
> Standardizing centers the data so that it has a zero mean and a standard deviation of 1, under the assumption
> that the data is normally distributed.
>
> * The distribution is relatively normal (aside from the white pixels which is highest in all classes).
* Using PCA could be useful since the dataset has a large amount of features. As previously stated,
PCA assumes the data has zero mean.
>
> Therefore, standard scaling is the ideal option.

In [ ]:
# create transformation pipeline

# How to change ColorConverter is_binary hyperparameter:
# full_pipeline['clr_convert'].__setattr__('is_binary', True)

full_pipeline = Pipeline([
    ('clr_convert', ColorConverter()),
    ('std_scaler', StandardScaler()),
])

In [ ]:
# transform training data using pipeline
X_train_prepared = full_pipeline.fit_transform(X_train)

# 4. Training and Evaluating Models

> As previously mentioned, the number of instances for each class in the training set are equal.
>
> Chosen evaluation metric:
>
> Accuracy works well with balanced classification tasks. It is also the most intuitive
> metric and is especially easier to understand when dealing with multiclass classification.
>> Note:
* Accuracy is sensitive to the test size. Therefore, I will use 6 cross-validation folds
(total of 10,000 instances in each test fold- matching the test set size).
* Since the features consist of pixel values (a fixed range), it is likely that the testing data would fit the
transformers nearly the same. This means that cross-validation should give good assessment
of the performance on unseen data, even when evaluating only from the fitted and transformed training data.

In [ ]:
# function for evaluating cross validation scores and for easy model comparison
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", round(scores.mean(), 5))
    print("Standard Deviation:", round(scores.std(), 5))

# function prints accuracy and errors
def display_evaluation(actual, pred, print_report=False, print_errors=False):
    print("Accuracy:", round(metrics.accuracy_score(actual, pred), 5))
    if print_report:
        print(metrics.classification_report(actual, pred))
    if print_errors:
        print("Confusion Matrix Errors:")
        conf_mx = metrics.confusion_matrix(actual, pred, labels=range(10))
        # confusion matrix rows represent actual class, columns represent predicted class
        row_sums = conf_mx.sum(axis=1, keepdims=True) # count sum of instances in each row (per class)
        norm_conf_mx = conf_mx / row_sums # divide confusion matrix by number of instances in each class
        np.fill_diagonal(norm_conf_mx, 0) # keep errors only by filling diagonal
        plt.matshow(norm_conf_mx, cmap=plt.cm.gray) # plot errors in confusion matrix
        plt.grid(False)
        plt.show()

### Shortlist Promising Models:
> Trying many models quickly and selecting the ones that show promising results.
>
>> How I plan to do this:
1. Train a baseline model and evaluate sample predictions.
2. Use cross-validation and evaluate the scores.
3. If the model has a significant hyperparameter, try changing it. Use cross-validation to
compare the results to step 2.

In [ ]:
# a few instances from the training data for testing
some_data = X_train.iloc[:10]
some_labels = y_train.iloc[:10]
some_data_prepared = full_pipeline.transform(some_data)

### Dummy Classifier

In [ ]:
dummy_clf = DummyClassifier(strategy="stratified", random_state=42)  # stratified uses training set class distribution
dummy_clf.fit(X_train_prepared, y_train)
dummy_clf_pred = cross_val_predict(dummy_clf, X_train_prepared, y_train, cv=6)
display_evaluation(y_train, dummy_clf_pred, True, True)

### 1. Logistic Regression: Grayscale

In [ ]:
# multinomial = softmax regression, used for multiclass
# softmax normalizes the total probability to a sum of 1
log_reg = LogisticRegression(multi_class='multinomial', random_state=42, n_jobs=-1)
# log_reg.fit(X_train_prepared, y_train)
# joblib.dump(log_reg, "FMNIST/models/log_reg_1.pkl")
log_reg = joblib.load(r"FMNIST/models/log_reg_1.pkl")

print("Predictions:", log_reg.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# log_reg_scores_1 = cross_val_score(log_reg, X_train_prepared, y_train, scoring='accuracy', cv=6)
# joblib.dump(log_reg_scores_1, "FMNIST/scores/log_reg_scores_1.pkl")
log_reg_scores_1 = joblib.load(r"FMNIST/scores/log_reg_scores_1.pkl")
display_scores(log_reg_scores_1)

### 2. KNN: Grayscale

In [ ]:
# using default weights (uniform)
# using default n_neighbors (n = 5)
knn_clf = KNeighborsClassifier(n_jobs=-1)
# knn_clf.fit(X_train_prepared, y_train)
# joblib.dump(knn_clf, "FMNIST/models/knn_clf_2.pkl")
knn_clf = joblib.load(r"FMNIST/models/knn_clf_2.pkl")

print("Predictions:", knn_clf.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# knn_scores_2 = cross_val_score(knn_clf, X_train_prepared, y_train, scoring='accuracy', cv=6)
# joblib.dump(knn_scores_2, "FMNIST/scores/knn_scores_2.pkl")
knn_scores_2 = joblib.load(r"FMNIST/scores/knn_scores_2.pkl")
display_scores(knn_scores_2)

### 3. Decision Tree Classifier: Grayscale

In [ ]:
tree_clf = DecisionTreeClassifier(random_state=42)
# tree_clf.fit(X_train_prepared, y_train)
# joblib.dump(tree_clf, "FMNIST/models/tree_clf_3.pkl")
tree_clf = joblib.load(r"FMNIST/models/tree_clf_3.pkl")

print("Predictions:", tree_clf.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# tree_scores_3 = cross_val_score(tree_clf, X_train_prepared, y_train, scoring='accuracy', cv=6, n_jobs=-1)
# joblib.dump(tree_scores_3, "FMNIST/scores/tree_scores_3.pkl")
tree_scores_3 = joblib.load(r"FMNIST/scores/tree_scores_3.pkl")
display_scores(tree_scores_3)

### 4. Random Forest Classifier: Grayscale

In [ ]:
rf_clf = RandomForestClassifier(random_state=42, n_jobs=-1)
# rf_clf.fit(X_train_prepared, y_train)
# joblib.dump(rf_clf, "FMNIST/models/rf_clf_4.pkl")
rf_clf = joblib.load(r"FMNIST/models/rf_clf_4.pkl")

print("Predictions:", rf_clf.predict(some_data_prepared))
print("Labels:", list(some_labels))

In [ ]:
# rf_scores_4 = cross_val_score(rf_clf, X_train_prepared, y_train, scoring='accuracy', cv=6)
# joblib.dump(rf_scores_4, "FMNIST/scores/rf_scores_4.pkl")
rf_scores_4 = joblib.load(r"FMNIST/scores/rf_scores_4.pkl")
display_scores(rf_scores_4)

> ***Before continuing, I will try these 4 models again on the black and white dataset.***

In [ ]:
# change hyperparameter to black and white images
full_pipeline['clr_convert'].__setattr__('is_binary', True)
full_pipeline.steps[0] # check that the hyperparameter changed

In [ ]:
# transform training data using pipeline
X_train_prepared2 = full_pipeline.fit_transform(X_train)

# a few instances from the training data for testing
some_data_prepared2 = full_pipeline.transform(some_data)

> Note: Fitting the same model will overwrite the previous fit.

### 5. Logistic Regression: Black and White

In [ ]:
# log_reg.fit(X_train_prepared2, y_train)
# joblib.dump(log_reg, "FMNIST/models/log_reg_5.pkl")
log_reg = joblib.load(r"FMNIST/models/log_reg_5.pkl")

print("Predictions:", log_reg.predict(some_data_prepared2))
print("Labels:", list(some_labels))

In [ ]:
# log_reg_scores_5 = cross_val_score(log_reg, X_train_prepared2, y_train, scoring='accuracy', cv=6)
# joblib.dump(log_reg_scores_5, "FMNIST/scores/log_reg_scores_5.pkl")
log_reg_scores_5 = joblib.load(r"FMNIST/scores/log_reg_scores_5.pkl")
display_scores(log_reg_scores_5)

### 6. KNN: Black and White

In [ ]:
# knn_clf.fit(X_train_prepared2, y_train)
# joblib.dump(knn_clf, "FMNIST/models/knn_clf_6.pkl")
knn_clf = joblib.load(r"FMNIST/models/knn_clf_6.pkl")

print("Predictions:", knn_clf.predict(some_data_prepared2))
print("Labels:", list(some_labels))

In [ ]:
# knn_scores_6 = cross_val_score(knn_clf, X_train_prepared2, y_train, scoring='accuracy', cv=6)
# joblib.dump(knn_scores_6, "FMNIST/scores/knn_scores_6.pkl")
knn_scores_6 = joblib.load(r"FMNIST/scores/knn_scores_6.pkl")
display_scores(knn_scores_6)

### 7. Decision Tree Classifier: Black and White

In [ ]:
# tree_clf.fit(X_train_prepared2, y_train)
# joblib.dump(tree_clf, "FMNIST/models/tree_clf_7.pkl")
tree_clf = joblib.load(r"FMNIST/models/tree_clf_7.pkl")

print("Predictions:", tree_clf.predict(some_data_prepared2))
print("Labels:", list(some_labels))

In [ ]:
# tree_scores_7 = cross_val_score(tree_clf, X_train_prepared2, y_train, scoring='accuracy', cv=6, n_jobs=-1)
# joblib.dump(tree_scores_7, "FMNIST/scores/tree_scores_7.pkl")
tree_scores_7 = joblib.load(r"FMNIST/scores/tree_scores_7.pkl")
display_scores(tree_scores_7)

### 8. Random Forest Classifier: Black and White

In [ ]:
# rf_clf.fit(X_train_prepared2, y_train)
# joblib.dump(rf_clf, "FMNIST/models/rf_clf_8.pkl")
rf_clf = joblib.load(r"FMNIST/models/rf_clf_8.pkl")

print("Predictions:", rf_clf.predict(some_data_prepared2))
print("Labels:", list(some_labels))

In [ ]:
# rf_scores_8 = cross_val_score(rf_clf, X_train_prepared2, y_train, scoring='accuracy', cv=6)
# joblib.dump(rf_scores_8, "FMNIST/scores/rf_scores_8.pkl")
rf_scores_8 = joblib.load(r"FMNIST/scores/rf_scores_8.pkl")
display_scores(rf_scores_8)

> #### Observations:
* All models performed slightly worse after converting the images to black and white, the mean after cross-validation
was lower.

> ### PCA
>
> PCA is a dimensionality reduction technique that projects the data onto a lower-dimensional hyperplane.
> The ideal projection is one that preserves most of the variance with the least amount of
> information loss. There are various ways to select the number of principal components/dimensions,
> such as plotting the explained variance or measuring the performance of a Machine Learning
> algorithm if PCA is used for pre-processing.
>
> Evaluating the models on both grayscale and black and white images was a vital step. While PCA does improve
> training computation time, some data is lost, so it is better to choose the transformation that has better results.
>
> Therefore, I will use the grayscale images for PCA.
>
>> #### PCA vs. K-Means:
> PCA works well with continuous values such as image pixels.
> Furthermore, K-Means clustering is an unsupervised learning technique which outputs new labels according to
> the assigned clusters. Since the data labels are specific (i.e. the clothing could have been split into
> 5 categories instead of 10: tops, bottoms, dresses, footwear, bags), clustering does not seem to be the best approach
> in this case.

In [ ]:
# change hyperparameter to grayscale images
full_pipeline['clr_convert'].__setattr__('is_binary', False)
full_pipeline.steps[0]  # check that the hyperparameter changed

In [ ]:
pca = PCA()
pca.fit(X_train_prepared)
# cumulative sum (increasing by sequential addition) of components
cumsum = np.cumsum(pca.explained_variance_ratio_)
d = np.argmax(cumsum >= 0.95) + 1
d

In [ ]:
plt.axis([0, 784, 0, 1])  # axis limits
plt.plot(cumsum, linewidth=3)
plt.xlabel("Dimensions", size=15)
plt.ylabel("Explained Variance", size=15)
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.tight_layout()
plt.show()

> #### Choosing the Number of Dimensions:
> Using the elbow method, around 95% explained variance is where the graph starts growing slower.
> For instance, reducing the dimensions from 784 down to 256 will preserve 95% of the variance with only
> a little data loss.
>
> I will use pipelines and cross-validation to determine the optimal explained variance value.

### 10. Logistic Regression: Grayscale, PCA

> Before PCA:

In [ ]:
display_scores(log_reg_scores_1)

> After PCA:

In [ ]:
pca_log_reg = Pipeline([
    ("pca", PCA()),
    ("log_reg", LogisticRegression(multi_class='multinomial', random_state=42))
])

param_grid = [{
    "pca__n_components": [0.9, 0.91, 0.92, 0.93, 0.94, 0.95]
}]

In [ ]:
# pca_log_reg_grid = GridSearchCV(pca_log_reg, param_grid, cv=6, scoring="accuracy", verbose=2, n_jobs=-1)
# pca_log_reg_grid.fit(X_train_prepared, y_train)
# joblib.dump(pca_log_reg_grid, "FMNIST/scores/pca_log_reg_grid.pkl")
pca_log_reg_grid = joblib.load(r"FMNIST/scores/pca_log_reg_grid.pkl")

In [ ]:
pca_log_reg_grid.best_params_

In [ ]:
pca_log_reg_grid.best_score_

In [ ]:
# show results for each iteration
cvres = pca_log_reg_grid.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

### 11. Random Forest: Grayscale, PCA

> Random Forest before PCA:

In [ ]:
display_scores(rf_scores_4)

> Random Forest after PCA:

In [ ]:
pca_rf_clf = Pipeline([
    ("pca", PCA()),
    ("rf_clf", RandomForestClassifier(random_state=42, n_jobs=-1))
])

In [ ]:
# pca_rf_clf_grid = GridSearchCV(pca_rf_clf, param_grid, cv=6, scoring="accuracy", verbose=2)
# pca_rf_clf_grid.fit(X_train_prepared, y_train)
# joblib.dump(pca_rf_clf_grid, "FMNIST/scores/pca_rf_clf_grid.pkl")
pca_rf_clf_grid = joblib.load(r"FMNIST/scores/pca_rf_clf_grid.pkl")

In [ ]:
pca_rf_clf_grid.best_params_

In [ ]:
pca_rf_clf_grid.best_score_

In [ ]:
# show results for each iteration
cvres = pca_rf_clf_grid.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> #### Observations:
* Logistic Regression had the highest score with 0.95 explained variance, slightly higher than before PCA.
* Random Forest had the highest score with 0.92 explained variance, the score is roughly 2% lower than before PCA.
* The Random Forest scores were all nearly the same. Meanwhile, the Logistic Regression scores show a downward trend.
Choosing 0.91 explained variance seems like a reasonable ratio, preserving most of the data and model accuracy.

In [ ]:
d = np.argmax(cumsum >= 0.91) + 1
d

> PCA reduces the dimensions from 784 to 153!

In [ ]:
# add PCA to pipeline
full_pipeline.steps.append(('pca', PCA(n_components=0.91)))
full_pipeline

In [ ]:
# transform data with reduced dimensions
X_train_reduced = full_pipeline.fit_transform(X_train, y_train)
some_data_reduced = full_pipeline.transform(some_data)

### 12. KNN: Grayscale, PCA

In [ ]:
# knn_scores_12 = cross_val_score(clone(knn_clf), X_train_reduced, y_train, cv=6, scoring='accuracy', verbose=2)
# joblib.dump(knn_scores_12, "FMNIST/scores/knn_scores_12.pkl")
knn_scores_12 = joblib.load(r"FMNIST/scores/knn_scores_12.pkl")
display_scores(knn_scores_12)

### 13. Decision Tree: Grayscale, PCA

In [ ]:
# tree_scores_13 = cross_val_score(clone(tree_clf), X_train_reduced, y_train, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# joblib.dump(tree_scores_13, "FMNIST/scores/tree_scores_13.pkl")
tree_scores_13 = joblib.load(r"FMNIST/scores/tree_scores_13.pkl")
display_scores(tree_scores_13)

### 14. Extra Trees Classifier: Grayscale, PCA
> Very similar to the Random Forest classifier. The main difference is that Extra Trees randomly splits
> nodes, while Random Forest searches for the best split. Extra Trees is also faster to train due to this.
>> Note: Comparing prediction accuracy can help determine which model performs better on the data.

In [ ]:
ex_trees = ExtraTreesClassifier(random_state=42, n_jobs=-1)
# ex_trees.fit(X_train_reduced, y_train)
# joblib.dump(ex_trees, "FMNIST/models/ex_trees.pkl")
ex_trees = joblib.load(r"FMNIST/models/ex_trees.pkl")

print("Predictions:", ex_trees.predict(some_data_reduced))
print("Labels:", list(some_labels))

In [ ]:
# ex_trees_scores_14 = cross_val_score(ex_trees, X_train_reduced, y_train, cv=6, scoring='accuracy', verbose=2)
# joblib.dump(ex_trees_scores_14, "FMNIST/scores/ex_trees_scores_14.pkl")
ex_trees_scores_14 = joblib.load(r"FMNIST/scores/ex_trees_scores_14.pkl")
display_scores(ex_trees_scores_14)

> Comparing Extra Trees to Random Forest:

In [ ]:
# rf_clf_scores_14 = cross_val_score(clone(rf_clf), X_train_reduced, y_train, cv=6, scoring='accuracy', verbose=2)
# joblib.dump(rf_clf_scores_14, "FMNIST/scores/rf_clf_scores_14.pkl")
rf_clf_scores_14 = joblib.load(r"FMNIST/scores/rf_clf_scores_14.pkl")
display_scores(rf_clf_scores_14)

> #### Observations:
* Extra Trees has a higher accuracy and a lower standard deviation, making it more reliable.

### 15. AdaBoost: Grayscale, PCA
> The AdaBoost (adaptive boosting) ensemble method trains a weak classifier, then increases the
> weights of the misclassified instances and trains another weak classifier on the dataset with the
> updated weights.
>> Note: The sklearn default algorithm ("SAMME.R") is multiclass. SAMME.R uses class probabilities
>> and SAMME uses the predicted classes.

In [ ]:
ada_clf = AdaBoostClassifier(DecisionTreeClassifier(), random_state=42)
# ada_clf.fit(X_train_reduced, y_train)
# joblib.dump(ada_clf, "FMNIST/models/ada_clf_15.pkl")
ada_clf = joblib.load(r"FMNIST/models/ada_clf_15.pkl")

print("Predictions:", ada_clf.predict(some_data_reduced))
print("Labels:", list(some_labels))

In [ ]:
# ada_scores_15 = cross_val_score(ada_clf, X_train_reduced, y_train, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# joblib.dump(ada_scores_15, "FMNIST/scores/ada_scores_15.pkl")
ada_scores_15 = joblib.load(r"FMNIST/scores/ada_scores_15.pkl")
display_scores(ada_scores_15)

### 16. XGBoost: Grayscale, PCA
> The Gradient Boosting ensemble method adds predictors by fitting a weak classifier to the errors
> made by the previous classifiers, then making predictions on the combined classifiers.
>> Note: XGBoost (extreme gradient boosting) is a separate library from sklearn
> that contains an optimized version of gradient boosting.

In [ ]:
# xgb_clf = XGBClassifier(random_state=42, n_jobs=-1, eval_metric='merror', objective='multi:softmax',
#                        num_class=10, use_label_encoder=False)
# xgb_clf.fit(X_train_reduced, y_train)
# joblib.dump(xgb_clf, "FMNIST/models/xgb_clf_16.pkl")
xgb_clf = joblib.load(r"FMNIST/models/xgb_clf_16.pkl")

print("Predictions:", xgb_clf.predict(some_data_reduced))
print("Labels:", list(some_labels))

In [ ]:
xgb_clf.best_iteration

In [ ]:
params = {
    "objective": "multi:softmax",
    "num_class": 10,
    "eval_metric": "merror"  # merror rate = 1-accuracy, m stands for multiclass
}

# dmatrix is a data structure used by the XGBoost library that optimizes performance
# it is needed when using the XGBoost library cross-validation
dmat_train = DMatrix(X_train_reduced, y_train)

In [ ]:
# xgb_scores_16 = cv(params, dmat_train, num_boost_round=100, nfold=6,
#                   early_stopping_rounds=5, seed=42, metrics="merror", verbose_eval=True)
# joblib.dump(xgb_scores_16, "FMNIST/scores/xgb_scores_16.pkl")
xgb_scores_16 = joblib.load(r"FMNIST/scores/xgb_scores_16.pkl")

In [ ]:
curr_min = xgb_scores_16["test-merror-mean"].argmin()
curr_min

In [ ]:
# plot classification errors from cross-validation
plt.figure(figsize=(7, 5))
sns.lineplot(data=[xgb_scores_16["test-merror-mean"], xgb_scores_16["train-merror-mean"]])
plt.xlabel("number of estimators")
plt.ylabel("classification error rate")
plt.title("XGBoost Classification Errors", size=15)
plt.tight_layout()
plt.show()

In [ ]:
# best accuracy
1 - xgb_scores_16["test-merror-mean"][curr_min]

> #### Observations:
* So far, the minimum errors are with 100 estimators. The errors only decreased, so during hyperparameter
tuning larger parameters should be tested.

> #### Looking Back:
* The best models (from lowest to highest scores): Logistic Regression, KNN, Extra Trees and XGBoost.
* Decision Tree and AdaBoost models had the lowest performance, both below 80% accuracy.
>
> The pre-processing steps and baseline models have been analyzed.
> The next step is to find the finest hyperparameters for the best models.

### Fine-Tune Logistic Regression

In [ ]:
# predict accuracy and plot errors before tuning

# log_reg_pred = cross_val_predict(log_reg, X_train_reduced, y_train, cv=6, verbose=2, n_jobs=-1)
# joblib.dump(log_reg_pred, "FMNIST/pred/ log_reg_pred.pkl")
log_reg_pred = joblib.load(r"FMNIST/pred/ log_reg_pred.pkl")
display_evaluation(y_train, log_reg_pred)

> * When the Logistic Regression parameter 'multi_class' is set to 'multinomial', it is using Softmax Regression
which is optimized for multi-class inputs. Simply put, it estimates the probability that an instance belongs
to each class and predicts the highest one. The sum of all probabilities is equal to 1.
* Another approach: One-vs-One and One-vs-Rest wrapper methods.
>
> I'll search for the best hyperparameters, then try applying the best estimator with the wrapper methods.

> Tuning solver and penalty:

In [ ]:
# both sag and saga solvers work well with large datasets and multiclass
# they are a variation of gradient descent
# convergence is faster when the data is scaled

param_grid = [{
    "solver": ["sag", "saga"]  # sag = stochastic average gradient, saga = variant of sag solver
}]

In [ ]:
# log_reg = LogisticRegression(multi_class='multinomial', random_state=42)
# log_reg_cv = GridSearchCV(log_reg, param_grid, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# log_reg_cv.fit(X_train_reduced, y_train)
# joblib.dump(log_reg_cv, "FMNIST/scores/log_reg_cv.pkl") # save local copy
log_reg_cv = joblib.load(r"FMNIST/scores/log_reg_cv.pkl")

In [ ]:
# show the best score
log_reg_cv.best_score_

In [ ]:
# best estimator
log_reg2 = log_reg_cv.best_estimator_
log_reg2

In [ ]:
# show results for each iteration
cvres = log_reg_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
param_grid = [{
    "C": [0.001, 0.01, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]  # C controls regularization,
    # light penalty if close to 1.0, strong penalty if close to 0.0
}]

In [ ]:
# log_reg_cv2 = GridSearchCV(log_reg2, param_grid, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# log_reg_cv2.fit(X_train_reduced, y_train)
# joblib.dump(log_reg_cv2, "FMNIST/scores/log_reg_cv2.pkl")
log_reg_cv2 = joblib.load(r"FMNIST/scores/log_reg_cv2.pkl")

In [ ]:
# show the best score
log_reg_cv2.best_score_

In [ ]:
# best estimator
log_reg3 = log_reg_cv2.best_estimator_
log_reg3

In [ ]:
# show results for each iteration
cvres = log_reg_cv2.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> ### One-vs-Rest:
> In this method, a binary classifier is trained for each class (10 in this case). The highest decision
> score out of all the classifiers is the predicted class.

In [ ]:
ovr = OneVsRestClassifier(clone(log_reg3), n_jobs=-1)
# ovr_scores = cross_val_score(ovr, X_train_reduced, y_train, scoring='accuracy', cv=6, n_jobs=-1)
# joblib.dump(ovr_scores, "FMNIST/scores/ovr_scores.pkl")
ovr_scores = joblib.load(r"FMNIST/scores/ovr_scores.pkl")
display_scores(ovr_scores)

> ### One-vs-One:
> In this method, a binary classifier is trained for every pair of labels. Although more classifiers
> are trained using this method, each needs to be trained only on the instances that represent the two
> classes.

In [ ]:
ovo = OneVsOneClassifier(clone(log_reg3), n_jobs=-1)
# ovo_scores = cross_val_score(ovo, X_train_reduced, y_train, scoring='accuracy', cv=6, n_jobs=-1)
# joblib.dump(ovo_scores, "FMNIST/scores/ovo_scores.pkl")
ovo_scores = joblib.load(r"FMNIST/scores/ovo_scores.pkl")
display_scores(ovo_scores)

> **Final Logistic Regression model:** OvO Logistic Regression, using sag solver, l2 norm and 0.5 penalty.

In [ ]:
# predict accuracy and plot errors after tuning

# log_reg_final_pred = cross_val_predict(ovo, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(log_reg_final_pred, "FMNIST/pred/log_reg_final_pred.pkl")
log_reg_final_pred = joblib.load(r"FMNIST/pred/log_reg_final_pred.pkl")
display_evaluation(y_train, log_reg_final_pred, True)

### Fine-Tune KNN

In [ ]:
# predict accuracy and plot errors before tuning

# knn_clf_pred = cross_val_predict(knn_clf, X_train_reduced, y_train, cv=6, verbose=2, n_jobs=-1)
# joblib.dump(knn_clf_pred, "FMNIST/pred/knn_clf_pred.pkl")
knn_clf_pred = joblib.load(r"FMNIST/pred/knn_clf_pred.pkl")
display_evaluation(y_train, knn_clf_pred)

> KNN does not output probabilities, which means that it can only be used for hard voting.
In this case, One-vs-Rest, One-vs-One, and a 'soft' Voting Classifier can not be used.

> Tuning weights and penalty:

In [ ]:
param_grid = [{
    "weights": ['uniform', 'distance'],
    "p": [1, 2]
}]

In [ ]:
# knn_clf_cv = GridSearchCV(KNeighborsClassifier(), param_grid, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# knn_clf_cv.fit(X_train_reduced, y_train)
# joblib.dump(knn_clf_cv, "FMNIST/scores/knn_clf_cv.pkl")
knn_clf_cv = joblib.load(r"FMNIST/scores/knn_clf_cv.pkl")

In [ ]:
# show the best score
knn_clf_cv.best_score_

In [ ]:
# best estimator
knn_clf2 = knn_clf_cv.best_estimator_
knn_clf2

In [ ]:
# show results for each iteration
cvres = knn_clf_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> Tuning number of neighbors:
>> Note: There is an even number of classes. It is better to choose an odd number of neighbors to
>> prevent ties.

In [ ]:
param_grid = [{
    "n_neighbors": [3, 5, 7, 9, 11, 13, 15]
}]

In [ ]:
# knn_clf_cv2 = GridSearchCV(knn_clf2, param_grid, cv=6, scoring='accuracy', n_jobs=-1, verbose=2)
# knn_clf_cv2.fit(X_train_reduced, y_train)
# joblib.dump(knn_clf_cv2, "FMNIST/scores/knn_clf_cv2.pkl")
knn_clf_cv2 = joblib.load(r"FMNIST/scores/knn_clf_cv2.pkl")

In [ ]:
# show the best score
knn_clf_cv2.best_score_

In [ ]:
# best estimator
knn_clf3 = knn_clf_cv2.best_estimator_
knn_clf3

In [ ]:
# show results for each iteration
cvres = knn_clf_cv2.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> **Final KNN model:** Distance weights, l1 norm and 5 neighbors.
>> The final KNN model has a slightly higher accuracy than the final Logistic Regression model!

In [ ]:
# predict accuracy and plot errors after tuning

# knn_clf_final_pred = cross_val_predict(knn_clf3, X_train_reduced, y_train, cv=6, verbose=2, n_jobs=-1)
## joblib.dump(knn_clf_final_pred, "FMNIST/pred/knn_clf_final_pred.pkl")
knn_clf_final_pred = joblib.load(r"FMNIST/pred/knn_clf_final_pred.pkl")
display_evaluation(y_train, knn_clf_final_pred, True)

### Fine-Tune Extra Trees

In [ ]:
# predict accuracy and plot errors before tuning

# ex_trees_pred = cross_val_predict(ex_trees, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(ex_trees_pred, "FMNIST/pred/ex_trees_pred.pkl")
ex_trees_pred = joblib.load(r"FMNIST/pred/ex_trees_pred.pkl")
display_evaluation(y_train, ex_trees_pred)

> Tuning maximum number features per tree:
>> Note: If this hyperparameter is set to 'None', it evaluates all features before a split. When dealing with high
> dimensional data (such as image classification) it can be convenient to select a smaller amount
> of features.

In [ ]:
param_grid = [{
    "max_features": ['sqrt', 0.2, 0.5, 0.8, None]  # max features that are evaluated before splitting a node
}]

In [ ]:
# ex_trees_cv = GridSearchCV(ExtraTreesClassifier(random_state=42, n_jobs=-1), param_grid, cv=6,
#                         scoring='accuracy', verbose=2)
# ex_trees_cv.fit(X_train_reduced, y_train)
# joblib.dump(ex_trees_cv, "FMNIST/scores/ex_trees_cv.pkl")
ex_trees_cv = joblib.load(r"FMNIST/scores/ex_trees_cv.pkl")

In [ ]:
# show the best score
ex_trees_cv.best_score_

In [ ]:
# best estimator
ex_trees2 = ex_trees_cv.best_estimator_
ex_trees2

In [ ]:
# show results for each iteration
cvres = ex_trees_cv.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> Tuning number of trees with Bagging and Pasting:
>> Note: More estimators tend to improve accuracy at the expense of increasing computational time.
> Additionally, from a certain amount of trees the model performance does not change much.

In [ ]:
param_grid = [{
    "n_estimators": [50, 100, 200],  # number of trees in the forest
    "bootstrap": [True, False]  # sampling with or without replacement
    # True = bagging, False (sklearn default) = pasting
}]

In [ ]:
# ex_trees_cv2 = GridSearchCV(ex_trees2, param_grid, cv=6, scoring="accuracy", verbose=2)
# ex_trees_cv2.fit(X_train_reduced, y_train)
# joblib.dump(ex_trees_cv2, "FMNIST/scores/ex_trees_cv2.pkl")
ex_trees_cv2 = joblib.load(r"FMNIST/scores/ex_trees_cv2.pkl")

In [ ]:
# show the best score
ex_trees_cv2.best_score_

In [ ]:
# best estimator
ex_trees3 = ex_trees_cv2.best_estimator_
ex_trees3

In [ ]:
# show results for each iteration
cvres = ex_trees_cv2.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> **Final Extra Trees model:** 200 estimators, pasting method (sampling without replacement), and considers
> all features when splitting a node.

In [ ]:
# predict accuracy and plot errors after tuning

# ex_trees_final_pred = cross_val_predict(ex_trees3, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(ex_trees_final_pred, "FMNIST/pred/ex_trees_final_pred.pkl")
ex_trees_final_pred = joblib.load(r"FMNIST/pred/ex_trees_final_pred.pkl")
display_evaluation(y_train, ex_trees_final_pred, True)

### Fine-Tune XGBoost

In [ ]:
# predict accuracy and plot errors before tuning

# xgb_clf_pred = cross_val_predict(xgb_clf, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(xgb_clf_pred, "FMNIST/pred/xgb_clf_pred.pkl")
xgb_clf_pred = joblib.load(r"FMNIST/pred/xgb_clf_pred.pkl")
display_evaluation(y_train, xgb_clf_pred)

> Tuning the number of estimators:

In [ ]:
params = {
    "objective": "multi:softmax",
    "num_class": 10,
    "eval_metric": "merror"
}

In [ ]:
# xgb_scores_cv1 = cv(params, dmat_train, num_boost_round=200, nfold=6,
#                    early_stopping_rounds=5, seed=42, metrics="merror", verbose_eval=True)
# joblib.dump(xgb_scores_cv1, "FMNIST/scores/xgb_scores_cv1.pkl")
xgb_scores_cv1 = joblib.load(r"FMNIST/scores/xgb_scores_cv1.pkl")

In [ ]:
curr_min = xgb_scores_cv1["test-merror-mean"].argmin()
curr_min

In [ ]:
# plot classification errors from cross-validation
plt.figure(figsize=(7, 5))
sns.lineplot(data=[xgb_scores_cv1["test-merror-mean"], xgb_scores_cv1["train-merror-mean"]])
plt.xlabel("number of estimators")
plt.ylabel("classification error rate")
plt.title("XGBoost Classification Errors", size=15)
plt.tight_layout()
plt.show()

In [ ]:
# best accuracy
1 - xgb_scores_cv1["test-merror-mean"][curr_min]

> Tuning learning rate and tree depth:

In [ ]:
param_grid = [{
    "max_depth": [3, 4, 5, 6],  # maximum depth of a tree
    "learning_rate": [0.1, 0.3]  # controls the weights after every boost (eta)
}]

In [ ]:
# xgb_scores_cv2 = GridSearchCV(XGBClassifier(random_state=42, n_jobs=-1, eval_metric='merror', objective='multi:softmax',
#                        num_class=10, use_label_encoder=False, n_estimators=122), param_grid, cv=6, scoring='accuracy', verbose=2)
# xgb_scores_cv2.fit(X_train_reduced, y_train)
# joblib.dump(xgb_scores_cv2, "FMNIST/scores/xgb_scores_cv2.pkl")
xgb_scores_cv2 = joblib.load(r"FMNIST/scores/xgb_scores_cv2.pkl")

In [ ]:
# show the best score
xgb_scores_cv2.best_score_

In [ ]:
# best estimator
xgb_clf2 = xgb_scores_cv2.best_estimator_
xgb_clf2

In [ ]:
# show results for each iteration
cvres = xgb_scores_cv2.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
param_grid = [{
    "max_depth": [6, 8],  # maximum depth of a tree
    "learning_rate": [0.3, 0.5]  # controls the weights after every boost (eta)
}]

In [ ]:
# xgb_scores_cv3 = GridSearchCV(xgb_clf2, param_grid, cv=6, scoring='accuracy', verbose=2)
# xgb_scores_cv3.fit(X_train_reduced, y_train)
# joblib.dump(xgb_scores_cv3, "FMNIST/scores/xgb_scores_cv3.pkl")
xgb_scores_cv3 = joblib.load(r"FMNIST/scores/xgb_scores_cv3.pkl")

In [ ]:
# show the best score
xgb_scores_cv3.best_score_

In [ ]:
# best estimator
xgb_clf3 = xgb_scores_cv3.best_estimator_
xgb_clf3

In [ ]:
# show results for each iteration
cvres = xgb_scores_cv3.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

In [ ]:
param_grid = [{
    "max_depth": [8, 10],  # maximum depth of a tree
    "learning_rate": [0.3]  # controls the weights after every boost (eta)
}]

In [ ]:
# xgb_scores_cv4 = GridSearchCV(xgb_clf3, param_grid, cv=6, scoring='accuracy', verbose=2)
# xgb_scores_cv4.fit(X_train_reduced, y_train)
# joblib.dump(xgb_scores_cv4, "FMNIST/scores/xgb_scores_cv4.pkl")
xgb_scores_cv4 = joblib.load(r"FMNIST/scores/xgb_scores_cv4.pkl")

In [ ]:
# show the best score
xgb_scores_cv4.best_score_

In [ ]:
# best estimator
xgb_clf4 = xgb_scores_cv4.best_estimator_
xgb_clf4

In [ ]:
# show results for each iteration
cvres = xgb_scores_cv4.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(mean_score, params)

> **Final XGBoost model:** 0.3 learning rate, maximum tree depth 8.

In [ ]:
# predict accuracy and plot errors after tuning

# xgb_clf_final_pred = cross_val_predict(xgb_clf4, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(xgb_clf_final_pred, "FMNIST/pred/xgb_clf_final_pred.pkl")
xgb_clf_final_pred = joblib.load(r"FMNIST/pred/xgb_clf_final_pred.pkl")
display_evaluation(y_train, xgb_clf_final_pred, True)

### Voting Classifiers
> The voting classifier is an ensemble that aggregates the predictions of multiple classifiers.
> This often leads to higher prediction accuracy.
>> Note: This does not guarantee that the voting classifier will achieve higher accuracy.
> The classifiers were trained on the same data and might make similar errors, so it is preferable
> to use classifiers with diverse algorithms.

> ### Hard Voting Classifier:
> Predicts the class that received the most votes.

In [ ]:
voting_hard = VotingClassifier(
    estimators=[
        ('log_reg', clone(log_reg3)),
        ('knn', clone(knn_clf3)),
        ('ex_trees', clone(ex_trees3))],
    voting='hard')

In [ ]:
# voting_hard_scores = cross_val_score(voting_hard, X_train_reduced, y_train, cv=6, scoring="accuracy", verbose=2)
# joblib.dump(voting_hard_scores, "FMNIST/scores/voting_hard_scores.pkl")
voting_hard_scores = joblib.load(r"FMNIST/scores/voting_hard_scores.pkl")
display_scores(voting_hard_scores)

In [ ]:
# voting_hard_pred = cross_val_predict(voting_hard, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(voting_hard_pred, "FMNIST/pred/voting_hard_pred.pkl")
voting_hard_pred = joblib.load(r"FMNIST/pred/voting_hard_pred.pkl")
display_evaluation(y_train, voting_hard_pred)

> ### Soft Voting Classifier:
> Averages the predicted probabilities.
>> Note: This voting method is only possible when all classifiers can output predicted probabilities.

In [ ]:
voting_soft = VotingClassifier(
    estimators=[
        ('log_reg', clone(log_reg3)),
        ('ex_trees', clone(ex_trees3)),
        ('xgb', clone(xgb_clf4))],
    voting='soft')

In [ ]:
# voting_soft_scores = cross_val_score(voting_soft, X_train_reduced, y_train, cv=6, scoring="accuracy", verbose=2)
# joblib.dump(voting_soft_scores, "FMNIST/scores/voing_soft_scores.pkl")
voting_soft_scores = joblib.load(r"FMNIST/scores/voing_soft_scores.pkl")
display_scores(voting_soft_scores)

In [ ]:
# voting_soft_pred = cross_val_predict(voting_soft, X_train_reduced, y_train, cv=6, verbose=2)
# joblib.dump(voting_soft_pred, "FMNIST/pred/voting_soft_pred.pkl")
voting_soft_pred = joblib.load(r"FMNIST/pred/voting_soft_pred.pkl")
display_evaluation(y_train, voting_soft_pred)

> #### Observations:
* Soft voting achieved a slightly higher accuracy compared to hard voting.
* The final XGBoost model was still better than both voting classifiers.

### Overall, the model with the highest accuracy score was XGBoost!

> ### Last Attempt to Improve Accuracy With Data Augmentation:
>
> Data augmentation is a strategy that is used to create more data from the existing data.
For example, the images could be rotated. Using this method can add more variation to the data.
>
> As a last attempt to improve the classification accuracy, I will increase the amount of training samples.
For each image, I will create 4 new instances by shifting the images by one pixel in different
directions (left, right, top, bottom). Lastly, I will fit the new training instances to the best
model, XGBoost in this case, and evaluate the results.

In [ ]:
# using scipy multidimensional image processing shift function
# function shifts array
# fills the values beyond the edge with white pixels
# x, y values for shifting according to axis
def shift_image(image, x, y):
    img = np.array(image).reshape(28, 28)
    shifted_img = shift(img, [x, y], cval=0, mode="constant")
    return shifted_img.reshape(-1)

In [ ]:
# shift sample image by one pixel in each direction
sample_img = X_train.iloc[5]
img_shifted1 = shift_image(sample_img, -1, 0).reshape(28, 28)
img_shifted2 = shift_image(sample_img, 1, 0).reshape(28, 28)
img_shifted3 = shift_image(sample_img, 0, -1).reshape(28, 28)
img_shifted4 = shift_image(sample_img, 0, 1).reshape(28, 28)

# plot shifted images
fig, ax = plt.subplots(1, 5, figsize=(21, 6))
fig.suptitle("Image Augmentation Sample", size=20)
ax[0].imshow(np.array(sample_img).reshape(28, 28), cmap="binary")
ax[0].set_title("Original", size=15)
ax[1].imshow(img_shifted1, cmap="binary")
ax[1].set_title("Up", size=15)
ax[2].imshow(img_shifted2, cmap="binary")
ax[2].set_title("Down", size=15)
ax[3].imshow(img_shifted3, cmap="binary")
ax[3].set_title("Left", size=15)
ax[4].imshow(img_shifted4, cmap="binary")
ax[4].set_title("Right", size=15)
plt.tight_layout()
plt.show()

> As stated earlier, accuracy is sensitive to the test size. It would take a very long time to evaluate
before and after augmentation with cross-validation on test sizes of 10,000 instances (30 folds after augmentation!).
For this reason, I will create a validation set from the training instances and use it for evaluation.
>
> The validation set will include 10,000 instances, leaving 50,000 for training.

In [ ]:
# train and validation set stratified split
X, X_val, y, y_val = train_test_split(X_train, y_train, stratify=y_train, test_size=10000, random_state=42)

In [ ]:
# make sure that class distribution is equal
y.value_counts().sort_index()

In [ ]:
# DataFrame shape (rows, columns)
print("Training Set:", X.shape)
print("Validation Set:", X_val.shape)

> ### XGBoost Model Before Augmentation:

In [ ]:
X_prep = full_pipeline.fit_transform(X)
X_val_prep = full_pipeline.transform(X_val)

In [ ]:
# xgb_before = clone(xgb_clf4)
# xgb_before.fit(X_prep, y)
# joblib.dump(xgb_before, "FMNIST/models/xgb_before.pkl")
xgb_before = joblib.load(r"FMNIST/models/xgb_before.pkl")
# xgb_before_pred = xgb_before.predict(X_val_prep)
# joblib.dump(xgb_before_pred, "FMNIST/pred/xgb_before_pred.pkl")
xgb_before_pred = joblib.load(r"FMNIST/pred/xgb_before_pred.pkl")
display_evaluation(y_val, xgb_before_pred, True)

> ### XGBoost model after augmentation:

In [ ]:
# X_arr = np.array(X)
# y_arr = np.array(y)

# X_aug = [image for image in X_arr] # list containing training images arrays
# y_aug = [label for label in y_arr] # list containing label values

# for x, y in ((1, 0), (-1, 0), (0, 1), (0, -1)):
# for image, label in zip(X_arr, y_arr):
# X_aug.append(shift_image(image, x, y))
# y_aug.append(label)

# convert data from list to numpy array
# X_aug = np.array(X_aug)
# y_aug = np.array(y_aug)

# shuffle the training instances
# shuffle_idx = np.random.permutation(len(X_aug))
# X_aug = X_aug[shuffle_idx]
# y_aug = y_aug[shuffle_idx]

# joblib.dump(X_aug, "FMNIST/data/X_aug.pkl")
# joblib.dump(y_aug, "FMNIST/data/y_aug.pkl")
X_aug = joblib.load(r"FMNIST/data/X_aug.pkl")
y_aug = joblib.load(r"FMNIST/data/y_aug.pkl")

In [ ]:
# check augmented dataset shape
X_aug.shape

In [ ]:
# check augmented dataset class distribution (should be equal)
pd.Series(y_aug).value_counts().sort_index()

In [ ]:
# use pipeline to transform data
X_aug_prep = full_pipeline.fit_transform(X_aug, y_aug)
X_val_aug_prep = full_pipeline.transform(X_val)

In [ ]:
# xgb_after = clone(xgb_clf4)
# xgb_after.fit(X_aug_prep, y_aug)
# joblib.dump(xgb_after, "FMNIST/models/xgb_after.pkl")
xgb_after = joblib.load(r"FMNIST/models/xgb_after.pkl")
# xgb_after_pred = xgb_after.predict(X_val_aug_prep)
# joblib.dump(xgb_after_pred, "FMNIST/pred/xgb_after_pred.pkl")
xgb_after_pred = joblib.load(r"FMNIST/pred/xgb_after_pred.pkl")
display_evaluation(y_val, xgb_after_pred, True)

> ### Conclusion:
> **Data augmentation improved the accuracy score of the final model!**

> Fit model on whole augmented training set:

In [ ]:
# X_arr = np.array(X_train)
# y_arr = np.array(y_train)

# X_train_augmented = [image for image in X_arr] # list containing training images arrays
# y_train_augmented = [label for label in y_arr] # list containing label values

# for x, y in ((1, 0), (-1, 0), (0, 1), (0, -1)):
# for image, label in zip(X_arr, y_arr):
# X_train_augmented.append(shift_image(image, x, y))
# y_train_augmented.append(label)

# convert data from list to numpy array
# X_train_augmented = np.array(X_train_augmented)
# y_train_augmented = np.array(y_train_augmented)

# shuffle the training instances
# shuffle_idx = np.random.permutation(len(X_train_augmented))
# X_train_augmented = X_train_augmented[shuffle_idx]
# y_train_augmented = y_train_augmented[shuffle_idx]

# joblib.dump(X_train_augmented, "FMNIST/data/X_train_augmented.pkl")
# joblib.dump(y_train_augmented, "FMNIST/data/y_train_augmented.pkl")
X_train_augmented = joblib.load(r"FMNIST/data/X_train_augmented.pkl")
y_train_augmented = joblib.load(r"FMNIST/data/y_train_augmented.pkl")

In [ ]:
# check augmented dataset shape
X_train_augmented.shape

In [ ]:
# check augmented dataset class distribution (should be equal)
pd.Series(y_train_augmented).value_counts().sort_index()

In [ ]:
# use pipeline to transform data
X_train_aug_prepared = full_pipeline.fit_transform(X_train_augmented)

In [ ]:
# xgb_clf_final = clone(xgb_clf4)
# xgb_clf_final.fit(X_train_aug_prepared, y_train_augmented)
# joblib.dump(xgb_clf_final, "FMNIST/models/xgb_clf_final.pkl")
xgb_clf_final = joblib.load(r"FMNIST/models/xgb_clf_final.pkl")

# 5. Evaluating the Test Set

In [ ]:
# separate test set predictors and labels
X_test = test_set.drop("label", axis=1)
y_test = test_set["label"].copy()

In [ ]:
# transform test set
X_test_prepared = full_pipeline.transform(X_test)

In [ ]:
# evaluate test set predictions
final_predictions = xgb_clf_final.predict(X_test_prepared)
display_evaluation(y_test, final_predictions, True, True)

> #### Resources:
1. Fashion MNIST Dataset <a href="https://www.kaggle.com/zalando-research/fashionmnist"
> title="Kaggle">link</a>
2. Feature Scaling Article <a href="https://towardsdatascience.com/all-about-feature-scaling-bcc0ad75cb35"
> title="towardsdatascience">link</a>
3. PCA Article <a href="https://towardsdatascience.com/pca-is-not-feature-selection-3344fb764ae6"
> title="towardsdatascience">link</a>
4. Extra Trees and Random Forest Comparison <a href="https://quantdare.com/what-is-the-difference-between-extra-trees-and-random-forest/"
> title="quantdare">link</a>